In [ ]:
# Libs

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# remover warnings
import warnings
warnings.filterwarnings('ignore')

#tratar legenda dos gráficos de barra
from textwrap import wrap

# Setando o estilo dos gráficos
sns.set_theme(style='darkgrid')

In [ ]:
# função para formatar o label dos gráficos de barra

def labels_bar(lista_labels):
    return ['\n'.join(wrap(l,8)) for l in lista_labels]

In [ ]:
# função para anotar os valores de cada barra no gráfico

def annotate_bar(ax):
    for p in ax.patches:
        y = p.get_height()
        ax.annotate(f'{y}',xy=(p.get_x() + p.get_width()/2,y),
                    va= 'baseline', fontsize=12, color='black', xytext=(0,1),
                    textcoords='offset points', ha='center')
        

def annotate_bar_horizontal(ax):
    for p in ax.patches:
        x = p.get_width()
        ax.annotate(f'{int(x)}',xy=(x,p.get_y() + p.get_height()/2),
                    va= 'center', fontsize=12, color='black', xytext=(0,1),
                    textcoords='offset points', ha='left')
        

def annotate_lineplot(ax):

    for l in ax.lines:
        for i in range(len(l.get_xdata())):
            ax.annotate(s=f'{l.get_ydata()[i]:.0f}',
                        xy=(l.get_xdata()[i],l.get_ydata()[i]),
                        xytext=(0,1), ha='left',
                        textcoords='offset points', fontsize=12)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projeto Unicornio/Startups+in+2021+end.csv')

In [ ]:
df.head()

In [ ]:
#Renomeando a Primeira coluna e setando-a como index
df.rename(columns={'Unnamed: 0':'index'}, inplace=True)
df.set_index(keys='index', inplace=True)

In [ ]:
df.info()

In [ ]:
# Identificando a quantidade de Missing's e sua proporção (%) no DF
pd.DataFrame(zip(df.isna().sum(), df.isna().sum()/ df.value_counts().sum()*100), 
             columns=['count','proportion'], index=df.columns)

In [ ]:
df['Industry'].replace('Artificial intelligence','Artificial Intelligence', inplace=True)
df['Industry'].replace('Finttech','Fintech', inplace=True)

In [ ]:
# Identificando a quantidade de empresas por ramo de atividade 
atividade_comercial = pd.DataFrame(zip(df['Industry'].value_counts(), df['Industry'].value_counts(normalize=True)),
             columns=['Count', 'Proportion'], index=df['Industry'].value_counts().keys())

atividade_comercial

In [ ]:
# adcionando uma nova coluna no DF
df['Date Joined'] = pd.to_datetime(df['Date Joined'])
df['year'] = pd.DatetimeIndex(df['Date Joined']).year
df.head()

In [ ]:
df['Country'].unique()

In [ ]:
df['Country'].replace(to_replace=r'[,]$',value='',regex=True, inplace=True)

In [ ]:
df['Valuation ($B)'].replace(to_replace=r'^[$]',value='', regex=True,inplace=True)
df['Valuation ($B)'] = pd.to_numeric(df['Valuation ($B)'])

In [ ]:
unicornio_paises = pd.DataFrame(df.groupby(by=['year'])['Country']\
                                .value_counts())\
                                .rename(columns={'Country':'Count'})\
                                .reset_index()

unicornio_paises

In [ ]:
top_10 = unicornio_paises.groupby('Country')['Count'].sum()\
    .sort_values(ascending=False)\
    .reset_index()[:10]

top_10

In [ ]:
valor_por_paises = df.groupby(['Country'])['Valuation ($B)'].sum()\
    .sort_values(ascending=False)\
    .reset_index()

valor_por_paises.head(10)

In [ ]:
years = unicornio_paises.groupby('year')['Count'].sum().reset_index()
years

In [ ]:
labels_atividades = labels_bar(atividade_comercial.index)
labels_atividades

In [ ]:
company = ' '.join(df['Company'].tolist())

In [ ]:
empresas = WordCloud(
    width=300,
    height=200,
    background_color='white'
).generate(company)

In [ ]:
fig = plt.figure(figsize=(20,15))

fig.set_constrained_layout('w_pad')

gs = fig.add_gridspec(3,3)

ax0 = fig.add_subplot(gs[0,:])

sns.barplot(data=atividade_comercial, x=atividade_comercial.index, y='Count', 
            ax=ax0)

annotate_bar(ax0)

ax0.set(
    title='Empresas por Atividade',
    ylabel='Quantidade de empresas',
    yticks=[],
    ylim=(0,210),
    xticklabels=labels_atividades
)

ax01 = fig.add_subplot(gs[1:2,:1])

sns.barplot(data=top_10,y='Country', x='Count', orient='h',
            ax=ax01, dodge=False )

annotate_bar_horizontal(ax01)

ax01.set(
    title='TOP 10 - Unicórnios x Pais',
    xlabel='Quantidade',
    xticks=[],
    xlim=(0,np.max(top_10['Count']+100)),
    ylabel='',
    yticklabels=labels_bar(top_10['Country'].tolist())
)

ax02 = fig.add_subplot(gs[1,1:])

sns.lineplot(data=years, x='year', y='Count', ax=ax02, ls=':')

annotate_lineplot(ax02)

ax02.set(
    title='Unicórnios x Ano',
    xticks=years['year'].tolist(),
    ylabel='Quantidade Unicórnios gerados',
    yticks=[],
    ylim=(0, np.max(years['Count'])+50)
)

ax03 = fig.add_subplot(gs[2,:2])

sns.barplot(data=valor_por_paises[:10], x='Country', y='Valuation ($B)')

ax03.set(
    title='Valuation ($) x País',
    ylim=(0,np.max(valor_por_paises['Valuation ($B)'][:10])+100),
    yticks=[],
    xticklabels=labels_bar(valor_por_paises[:10]['Country'].tolist())
)

annotate_bar(ax03)

# Word Cloud
ax04 = fig.add_subplot(gs[2,2:])
ax04.imshow(empresas, interpolation='bilinear')
ax04.set_axis_off()

ax04.set(
    title = 'Company Names'
)

fig.suptitle('Projeto Análise Unicórnio - Magda Gomes',fontsize=20, y=1.05)

plt.show()